In [1]:
import os
import sys
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tdata
import numpy as np
from tqdm import tqdm
import cv2

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

from cancer_classifier.config import MODELS_DIR, RAW_DATA_DIR, PROCESSED_DATA_DIR, INTERIM_DATA_DIR, CLASSES
from cancer_classifier.processing.image_utils import adjust_image_contrast, resize_image_tensor, normalize_image_tensor, augment_image_tensor, process_dataset, save_processed_images, crop_image

%load_ext autoreload
%autoreload 2

2025-05-12 23:01:38.025 | INFO     | cancer_classifier.config:<module>:11 - PROJ_ROOT path is: /home/ayoubvip/deep-learning-cancer-classifier


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


## Data Preprocessing

### data crop and contract equilazing

In [4]:
for i, cls in enumerate(CLASSES):
    cls_dir = os.path.join(RAW_DATA_DIR, cls)
    for img_name in os.listdir(cls_dir):
        img_path = os.path.join(cls_dir, img_name)
        img = crop_image(img_path)
        # img = adjust_image_contrast(img)

        # save in processed directory
        cv2.imwrite(os.path.join(INTERIM_DATA_DIR, cls, img_name), img)
        # save_processed_images(img_tensor, os.path.join(PROCESSED_DATA_DIR, cls), img_name)

In [5]:
img_size = (256, 256)
batch_nbr = 16
train_ratio = 0.80
test_ratio = 0.10
val_ratio = 0.10

### data augmentation

In [ ]:
transformers = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=img_size),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.RandomRotation(10),
    # torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    torchvision.transforms.RandomResizedCrop(size=img_size, scale=(0.8, 1.0)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.5], std=[0.5])
])

dataset = torchvision.datasets.ImageFolder(root=RAW_DATA_DIR, transform=transformers, target_transform=PROCESSED_DATA_DIR)

### data spliting 

In [7]:
rand_gen = torch.Generator().manual_seed(142)
train_dataset, val_dataset, test_dataset = tdata.random_split(
    dataset = dataset,
    lengths=[train_ratio, val_ratio, test_ratio],
    generator=rand_gen
)

# Model Architecture: CNN

In [ ]:
def conv_block(in_channels, out_channels, kernel_size=3, stride=1, padding=1, pool_kernel_size=2, pool_stride=2, dropout_prob=0.2):
  return nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True),
      nn.Dropout2d(p=dropout_prob),
      nn.MaxPool2d(pool_kernel_size, pool_stride)
  )